# Table of Contents
1. [Introduction](#Introduction)

2. [Loading the libraries](#Loading-the-libraries)

2. [Loading the dataset](#Loading-the-dataset)

3. [Dataset Overview](#Dataset-Overview)

4. [Handling Missing Data](#Handling-Missing-Data)

5. [Feature Engineering](#Feature-Engineering)

6. [Data Visualization](#Data-Visualization)

7. [Conclusion](#Conclusion-and-Next-Steps)

# Introduction

### Business Problem Statement
The dataset consists of maintenance records from Coca-Cola Swire's production plants. The objective is to analyze maintenance durations, identify inefficiencies, and uncover insights to improve operational efficiency by minimizing unplanned maintenance and downtime.

### Objective of the EDA
In this exploratory data analysis (EDA) I have done the following:
1. Clean and preprocess the Coca-Cola Swire maintenance dataset, addressing missing values and outliers.
2. Analyze trends in planned and unplanned maintenance across plants and equipment.
3. Visualize insights to identify patterns, inefficiencies, and areas requiring process improvements for better resource allocation and predictive maintenance planning.









In [22]:
import warnings
warnings.filterwarnings("ignore")

# Loading the libraries

In [23]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Loading the dataset

In [24]:
# Load the dataset
file_path = "IWC_Work_Orders_Extract (1).csv"
data = pd.read_csv(file_path)

# Display basic information
print(data.info())
print(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1427264 entries, 0 to 1427263
Data columns (total 25 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   ORDER_ID                         1427264 non-null  int64  
 1   PLANT_ID                         1427264 non-null  object 
 2   PRODUCTION_LOCATION              1427264 non-null  object 
 3   EXECUTION_START_DATE             1427264 non-null  object 
 4   EXECUTION_FINISH_DATE            1427264 non-null  object 
 5   ACTUAL_START_TIME                1427264 non-null  object 
 6   ACTUAL_FINISH_TIME               1427264 non-null  object 
 7   ACTUAL_WORK_IN_MINUTES           1427264 non-null  float64
 8   MAINTENANCE_PLAN                 152594 non-null   object 
 9   MAINTENANCE_ITEM                 152594 non-null   float64
 10  MAINTENANCE_ACTIVITY_TYPE        1427264 non-null  object 
 11  ORDER_DESCRIPTION                291767 non-null  

The dataset contains over 1.4 million rows and 25 columns, with significant missing values in MAINTENANCE_PLAN and ORDER_DESCRIPTION. Time-based columns like EXECUTION_START_DATE offer potential for trend analysis, but missing data needs addressing to ensure accurate insights.

# Dataset Overview

In [3]:
# Check for missing values
print("Missing values per column:")
print(data.isnull().sum())

# Check for duplicates
print(f"Number of duplicate rows: {data.duplicated().sum()}")

# Data types and unique values
print("Column Data Types and Unique Values:")
print(data.nunique())

Missing values per column:
ORDER_ID                                 0
PLANT_ID                                 0
PRODUCTION_LOCATION                      0
EXECUTION_START_DATE                     0
EXECUTION_FINISH_DATE                    0
ACTUAL_START_TIME                        0
ACTUAL_FINISH_TIME                       0
ACTUAL_WORK_IN_MINUTES                   0
MAINTENANCE_PLAN                   1274670
MAINTENANCE_ITEM                   1274670
MAINTENANCE_ACTIVITY_TYPE                0
ORDER_DESCRIPTION                  1135497
MAINTENANCE_TYPE_DESCRIPTION       1135469
FUNCTIONAL_LOC                     1135482
FUNCTIONAL_AREA_NODE_1_MODIFIED    1137147
FUNCTIONAL_AREA_NODE_2_MODIFIED    1144857
FUNCTIONAL_AREA_NODE_3_MODIFIED    1168573
FUNCTIONAL_AREA_NODE_4_MODIFIED    1189691
FUNCTIONAL_AREA_NODE_5_MODIFIED    1425084
EQUIPMENT_ID                       1135469
EQUIPMENT_DESC                     1347817
EQUIP_CAT_DESC                     1347817
EQUIP_START_UP_DATE        

The dataset has no duplicates but significant missing values in many columns, with over 80% nulls in some. Key columns like ORDER_ID are unique, while PLANT_ID and PRODUCTION_LOCATION have low variability, ideal for grouping analysis.

In [4]:
# Summary statistics for numerical columns
print("Summary Statistics:")
print(data.describe())

# Analyze categorical columns
categorical_columns = data.select_dtypes(include='object').columns
for col in categorical_columns:
    print(f"Value counts for {col}:")
    print(data[col].value_counts())
    print("\n")

Summary Statistics:
           ORDER_ID  ACTUAL_WORK_IN_MINUTES  MAINTENANCE_ITEM  EQUIPMENT_ID
count  1.427264e+06            1.427264e+06     152594.000000  2.917950e+05
mean   4.561470e+10            8.862968e+01     115846.807954  3.002609e+08
std    8.387114e+10            8.037915e+02      71325.255687  5.581645e+06
min    1.000219e+08            0.000000e+00      39801.000000  2.000871e+08
25%    4.022602e+08            1.200000e+01      41046.000000  3.000092e+08
50%    7.044271e+08            4.800000e+01     112995.000000  3.000178e+08
75%    1.002343e+09            9.000000e+01     134702.000000  3.000269e+08
max    9.000001e+11            3.301848e+05     346352.000000  4.000290e+08
Value counts for PLANT_ID:
G261    630903
G221    290975
G291    225279
G816    116303
G811     84694
G812     79110
Name: PLANT_ID, dtype: int64


Value counts for PRODUCTION_LOCATION:
SILVERSTONE    630903
SUZUKA         290975
MONZA          225279
COTA           116303
MONACO          84694


Machines                      79224
Production resources/tools       82
Plant & Buildings                77
Test/measurement equipment       64
Name: EQUIP_CAT_DESC, dtype: int64


Value counts for EQUIP_START_UP_DATE:
2017-02-15    53650
2020-04-22     4270
2019-10-10     4115
2017-06-01     3018
2019-08-26     1971
              ...  
2024-05-16        2
2023-09-01        2
1960-01-01        1
2019-05-01        1
2021-01-04        1
Name: EQUIP_START_UP_DATE, Length: 92, dtype: int64


Value counts for EQUIP_VALID_FROM:
2017-02-15    32713
2017-02-25     6968
2019-10-06     4115
2023-12-09     2958
2017-10-30     2956
              ...  
2024-05-20        2
2021-09-01        1
2023-03-23        1
2021-01-26        1
2021-11-19        1
Name: EQUIP_VALID_FROM, Length: 137, dtype: int64


Value counts for EQUIP_VALID_TO:
9999-12-31    79447
Name: EQUIP_VALID_TO, dtype: int64




The summary statistics reveal that most maintenance tasks are completed on the same day (median and 25th percentile are 0). A small number of tasks take significantly longer, with a maximum duration of 1537 days, likely outliers.

# Handling Missing Data 

In [5]:
# Columns like FUNCTIONAL_AREA_NODE_5_MODIFIED and EQUIP_VALID_TO with more than 80% missing values might not be useful.
missing_threshold = 0.8
cols_to_drop = [col for col in data.columns if data[col].isnull().mean() > missing_threshold]
data = data.drop(columns=cols_to_drop)

## Imputing missing Values

In [6]:
# Example of filling with default values or placeholders
data['ORDER_DESCRIPTION'] = data['ORDER_DESCRIPTION'].fillna('Unknown')
data['MAINTENANCE_TYPE_DESCRIPTION'] = data['MAINTENANCE_TYPE_DESCRIPTION'].fillna('Not Specified')

1. Columns like FUNCTIONAL_AREA_NODE_5_MODIFIED and EQUIP_VALID_TO with more than 80% missing values were dropped. These columns provided limited value for analysis due to their high missing data proportion.
2. ORDER_DESCRIPTION was imputed with 'Unknown', ensuring that the missing entries are represented meaningfully without biasing the analysis.
3. MAINTENANCE_TYPE_DESCRIPTION was filled with 'Not Specified', preserving categorical integrity while reducing missing data.

# Feature Engineering

In [7]:
# Convert the maintenance type into binary or categorical values
data['Maintenance_Type_Binary'] = data['MAINTENANCE_TYPE_DESCRIPTION'].apply(lambda x: 1 if 'Planned' in str(x) else 0)

Converting maintenance types into binary helps in identifying trends or correlations between planned maintenance and other key metrics, such as task duration or frequency.

In [8]:
if 'EXECUTION_START_DATE' in data.columns:
    data['EQUIP_START_UP_DATE'] = pd.to_datetime(data['EXECUTION_START_DATE'], errors='coerce')
    data['Equipment_Age_Years'] = (pd.Timestamp.now() - data['EQUIP_START_UP_DATE']).dt.days // 365
else:
    print("No alternative date column found to calculate Equipment Age.")

In [9]:
print(data[['EQUIP_START_UP_DATE', 'Equipment_Age_Years']].head())

  EQUIP_START_UP_DATE  Equipment_Age_Years
0          2024-05-04                    0
1          2022-09-13                    2
2          2022-12-21                    1
3          2022-07-04                    2
4          2023-03-15                    1


Calculating equipment age allows us to examine whether older equipment requires more frequent or longer maintenance.

In [15]:
# Ensure the date columns are in datetime format
data['EXECUTION_START_DATE'] = pd.to_datetime(data['EXECUTION_START_DATE'], errors='coerce')
data['EXECUTION_FINISH_DATE'] = pd.to_datetime(data['EXECUTION_FINISH_DATE'], errors='coerce')

# Calculate the maintenance duration in days
data['Maintenance_Duration_Days'] = (data['EXECUTION_FINISH_DATE'] - data['EXECUTION_START_DATE']).dt.days

In [17]:
# Check for null values in the new column
print(data['Maintenance_Duration_Days'].isnull().sum())

# Fill null values with a placeholder (e.g., 0) or drop them
data['Maintenance_Duration_Days'] = data['Maintenance_Duration_Days'].fillna(0)


0


In [20]:
# Calculating equipment
data['EQUIP_START_UP_DATE'] = pd.to_datetime(data['EQUIP_START_UP_DATE'], errors='coerce')
data['Equipment_Age_Years'] = (pd.Timestamp.now() - data['EQUIP_START_UP_DATE']).dt.days // 365

In [21]:
print(data['Maintenance_Duration_Days'].describe())

count    1.427264e+06
mean     7.240048e-01
std      7.855836e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.537000e+03
Name: Maintenance_Duration_Days, dtype: float64


The high standard deviation (7.85 days) and maximum value (1,537 days) suggest the presence of outliers, indicating that some tasks are significantly delayed or represent long-term maintenance projects.


# Data Visualization

In [ ]:
# Boxplot for Maintenance Duration Days to detect outliers
plt.figure(figsize=(10, 6))
sns.boxplot(x=data['Maintenance_Duration_Days'], color='skyblue')
plt.title('Boxplot of Maintenance Duration Days')
plt.xlabel('Maintenance Duration (Days)')
plt.show()

The boxplot reveals the presence of significant outliers in the Maintenance_Duration_Days variable, as seen by the points extending far beyond the whiskers. These represent maintenance durations that are much longer than the typical values.

In [ ]:
# Histogram for Maintenance Duration Days to visualize distribution
plt.figure(figsize=(10, 6))
sns.histplot(data['Maintenance_Duration_Days'], bins=50, kde=True, color='blue')
plt.title('Histogram of Maintenance Duration Days')
plt.xlabel('Maintenance Duration (Days)')
plt.ylabel('Frequency')
plt.show()


The histogram highlights a highly skewed distribution for Maintenance_Duration_Days, with most values concentrated near zero. This confirms that maintenance durations are typically short, but there are a few extreme outliers that significantly extend the range.

In [ ]:
# Check top outliers for Maintenance_Duration_Days
top_outliers = data[data['Maintenance_Duration_Days'] > 365]  # Example: Maintenance lasting over a year
print(top_outliers[['ORDER_ID', 'Maintenance_Duration_Days', 'MAINTENANCE_TYPE_DESCRIPTION']])

Some maintenance orders exceed 365 days, indicating potential data entry errors or delays. Many long-duration orders are labeled as "Not Specified," suggesting incomplete records.A few long-duration corrective orders may highlight systemic inefficiencies.

In [ ]:
## Correlational Analysis

In [ ]:
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()

1. Most features show weak correlations, indicating limited linear relationships.
2. Moderate correlation (~0.23) between ACTUAL_WORK_IN_MINUTES and Maintenance_Duration_Days.
3. Minimal influence of Equipment_Age_Years on other variables.
4. Maintenance_Type_Binary has negligible correlations, suggesting non-linear or categorical factors at play.

In [ ]:
# Scatterplot: Work Minutes vs Maintenance Duration
sns.scatterplot(x='ACTUAL_WORK_IN_MINUTES', y='Maintenance_Duration_Days', data=data)
plt.title("Actual Work Minutes vs Maintenance Duration")
plt.show()

Cluster near the origin: A majority of data points are concentrated near lower values for both features, indicating that most maintenance tasks are completed within a shorter duration and require fewer work minutes.
Outliers: A few extreme values are visible, suggesting some tasks required significantly more work minutes or lasted much longer in duration, which could indicate special cases or data anomalies.
No strong linear relationship: The points appear scattered, implying a weak or no direct correlation between work minutes and maintenance duration.


In [ ]:
sns.boxplot(x='Maintenance_Type_Binary', y='Maintenance_Duration_Days', data=data)
plt.title("Maintenance Duration by Maintenance Type")
plt.show()

In [ ]:
# Calculate the 90th percentile of Maintenance_Duration_Days
percentile_90 = data['Maintenance_Duration_Days'].quantile(0.90)

# Filter rows with Maintenance_Duration_Days above the 90th percentile
outliers = data[data['Maintenance_Duration_Days'] > percentile_90]

# Print summary of outliers
print(f"Number of outliers: {len(outliers)}")
print(outliers[['ORDER_ID', 'Maintenance_Duration_Days', 'MAINTENANCE_TYPE_DESCRIPTION', 'Equipment_Age_Years']].head())


There are 62,035 rows where the Maintenance_Duration_Days exceeds the 90th percentile. This represents significant outliers that need to be reviewed.

In [ ]:
# Visualize the distribution of outliers
plt.figure(figsize=(10, 6))
sns.histplot(outliers['Maintenance_Duration_Days'], kde=True, color='red')
plt.title("Distribution of Outliers in Maintenance Duration Days")
plt.xlabel("Maintenance Duration (Days)")
plt.ylabel("Frequency")
plt.show()

 Most outliers fall between 0 to 200 days, suggesting that even among outliers, durations longer than 200 days are rare. A few extreme values extend beyond 1,000 days, highlighting rare but significant anomalies in the dataset. The distribution is highly skewed, which is typical for maintenance data, where a majority of tasks have shorter durations while a few extend significantly longer.

In [ ]:
# Boxplot for Maintenance Duration by Plant
sns.boxplot(x='PLANT_ID', y='Maintenance_Duration_Days', data=data)
plt.title("Maintenance Duration by Plant ID")
plt.show()

Most plants have a similar median maintenance duration (likely low).Significant outliers exist in all plants, particularly noticeable in G221 and G291, indicating prolonged maintenance durations for specific cases.
While the distribution is largely consistent across plants, a few plants have higher spread and extreme values, hinting at potential operational inefficiencies.

In [ ]:
# Bar chart for Maintenance Type
data.groupby('MAINTENANCE_ACTIVITY_TYPE')['Maintenance_Duration_Days'].mean().plot(kind='bar')
plt.title("Average Maintenance Duration by Activity Type")
plt.show()

1. For Unplanned Maintenance:
Takes significantly longer on average compared to planned maintenance.
Indicates potential inefficiencies or unexpected challenges during unplanned repairs.
2. For Planned Maintenance:
Much shorter average duration, suggesting better resource allocation and preparation.

In [ ]:
# Filter unplanned maintenance records
unplanned_maintenance = data[data['MAINTENANCE_ACTIVITY_TYPE'] == 'Unplanned']

# Check basic statistics for unplanned maintenance durations
print(unplanned_maintenance['Maintenance_Duration_Days'].describe())

# Top contributing equipment or plants for unplanned maintenance
top_unplanned_plants = unplanned_maintenance['PLANT_ID'].value_counts().head(5)
top_unplanned_equipment = unplanned_maintenance['EQUIPMENT_ID'].value_counts().head(5)

print("Top Plants with Unplanned Maintenance:\n", top_unplanned_plants)
print("Top Equipment IDs with Unplanned Maintenance:\n", top_unplanned_equipment)

The average unplanned maintenance duration is approximately 0.78 days, with a standard deviation of 8.27 days. The highest number of unplanned maintenance tasks occurred at Plant G261, followed by G221 and G291.

In [ ]:
# Group data by plant and calculate mean duration for unplanned maintenance
avg_unplanned_by_plant = unplanned_maintenance.groupby('PLANT_ID')['Maintenance_Duration_Days'].mean().sort_values(ascending=False)

# Plot average maintenance durations for unplanned maintenance by plant
avg_unplanned_by_plant.plot(kind='bar', figsize=(10, 6), title='Average Unplanned Maintenance Duration by Plant')
plt.ylabel('Average Duration (Days)')
plt.xlabel('Plant ID')
plt.show()

The bar chart shows the average unplanned maintenance duration by plant. The key insights are:
1. G291 and G221: These plants have the highest average unplanned maintenance durations, exceeding 1 day on average. These plants may require targeted maintenance strategies or operational improvements.
2. G816 and G812: These plants exhibit significantly shorter average maintenance durations, indicating more efficient maintenance processes or fewer severe issues.

In [ ]:
# Filter records with high maintenance durations (e.g., top 10% quantile)
high_duration_threshold = unplanned_maintenance['Maintenance_Duration_Days'].quantile(0.9)
high_duration_unplanned = unplanned_maintenance[unplanned_maintenance['Maintenance_Duration_Days'] > high_duration_threshold]

# Analyze equipment responsible for long unplanned maintenance durations
high_duration_equipment = high_duration_unplanned['EQUIPMENT_ID'].value_counts().head(5)
print("Top Equipment Causing Long Unplanned Maintenance Durations:\n", high_duration_equipment)

The equipment IDs listed above are responsible for the highest numbers of long unplanned maintenance durations

# Conclusion
This exploratory data analysis reveals significant insights into the dataset:

Key Findings:
1. Maintenance durations are generally short, but extreme outliers (e.g., 1,537 days) indicate inefficiencies or data inconsistencies.
2. Unplanned maintenance takes significantly longer on average than planned maintenance, highlighting operational challenges.
3. Plants like G291 and G221 and equipment like ID 300017654 are top contributors to prolonged unplanned maintenance durations.
4. Equipment age doesn’t show a strong linear correlation with maintenance duration but may still influence certain unplanned tasks.

Operational Insights:
1. G261, G221, and G291 have the highest volume of unplanned maintenance, which suggests areas to prioritize for resource allocation and process improvements.
2. Preventive maintenance appears effective, as planned tasks generally have shorter durations.


## What Can We Do Further?
Predictive Maintenance Modeling: we can predict which equipment or plant is likely to require unplanned maintenance or experience long durations by building a predictive model.
